In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

import torch
from torch_geometric.nn import GAE
from torch_geometric.utils import is_undirected, to_undirected


from torch.utils.data import DataLoader, TensorDataset

from model.GCN_GRU import GCN_GRU
from utils import *

In [2]:
shop_col = 'stonc_6_label'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
embedding_dim = 64
entity_dim = 8
epochs = 50
learning_rate = 0.0001

weights_path = 'GCN_GRU_Link_Prediction'

In [3]:
data_path = './data'

chid_dict_file = 'sample/sample_50k_idx_map.npy'
cdtx_file = 'sample/sample_50k_cdtx.csv'
cust_file = 'preprocessed/df_cust_log.csv'

chid_path = os.path.join(data_path, chid_dict_file)
cdtx_path = os.path.join(data_path, cdtx_file)
cust_path = os.path.join(data_path, cust_file)

In [4]:
df_cdtx, df_cust, n_users, n_shops = read_sample_files(cdtx_path,
                                                       cust_path,
                                                       chid_path,
                                                       shop_col)

Start reading cdtx file...
Finish reading cdtx file !
Start reading cust file...
Finish reading cust file !
Start mapping encoding...
Finish !!


In [5]:
list_months = sorted(df_cdtx.csmdt.unique())

ignore_cols = ['chid', 'data_dt']
category_cols = [f'category_{i+1}' for i in range(6)]
numeric_cols = list(set(df_cust.columns) - set(category_cols) - set(ignore_cols))

emb_dims = list(df_cust[category_cols].nunique())

input_dim = len(category_cols)*entity_dim + len(numeric_cols)

In [11]:
edge_dict = {}
edge_weights_dict = {}
max_objam = np.log1p(df_cdtx.groupby(['chid', shop_col]).objam.sum().max())
for month in tqdm(list_months):
    edges = df_cdtx[df_cdtx.csmdt==month].groupby(['chid', shop_col]).objam.sum()
    edge_pairs = np.stack([np.array(i) for i in edges.index]).T
    edge_pairs = torch.LongTensor(edge_pairs)
    
    edge_weights = np.log(edges.values+1)/max_objam
    edge_weights = torch.Tensor(edge_weights)
    
    if not is_undirected(edge_pairs):
        edge_pairs = torch.cat([edge_pairs, edge_pairs[[1,0],:]], -1)
        edge_weights = edge_weights.repeat(2)
    
    edge_dict[month] = edge_pairs
    edge_weights_dict[month] = edge_weights

100%|██████████| 25/25 [00:20<00:00,  1.23it/s]


In [13]:
# sample negative edges
# neg_edge_dict = {}
# for month in tqdm(list_months):
#     neg_edge_dict[month] = sample_neg_edges(edge_dict[month], n_users+n_shops, n_users, sampling_ratio=10)

# Load negative edges
neg_edge_dict = np.load('negative_edges_10.npy', allow_pickle=True).item()

In [14]:
model = GAE(GCN_GRU(input_dim, embedding_dim, entity_dim, emb_dims)).to(device)

x_num = []
x_cat = []
for i in list_months:
    cust_num_features = df_cust[df_cust.data_dt==i][numeric_cols].to_numpy()
    cust_num_features = torch.Tensor(cust_num_features)
    cust_cat_features = df_cust[df_cust.data_dt==i][category_cols].to_numpy()
    cust_cat_features = torch.LongTensor(cust_cat_features)
    
    shop_num_features = torch.zeros(n_shops, cust_num_features.shape[1])
    shop_cat_features = torch.zeros(n_shops, cust_cat_features.shape[1]).long()
    
    x_num.append(torch.cat([cust_num_features, shop_num_features], 0).to(device))
    x_cat.append(torch.cat([cust_cat_features, shop_cat_features], 0).to(device))

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
pos_edge_index = [i.to(device) for i in edge_dict.values()]
neg_edge_index = [i.to(device) for i in neg_edge_dict.values()]
pos_edge_weigts = [i.to(device) for i in edge_weights_dict.values()]

In [15]:
def train():
    model.train()
    loss_ = 0
    for i in tqdm(range(10)):
        optimizer.zero_grad()
        z = model.encode(x_cat[i:i+12], x_num[i:i+12], pos_edge_index[i:i+12], pos_edge_weigts[i:i+12])
        loss = model.recon_loss(z, pos_edge_index[i+12], neg_edge_index[i+12])
        loss_ += loss.item()
        loss.backward()
        optimizer.step()
    return loss_/10

In [16]:
def test(pos_edge_index, neg_edge_index):
    model.eval()
    total_auc = 0
    total_ap = 0
    
    for i in range(10,12):

        with torch.no_grad():
            z = model.encode(x_cat[i:i+12], x_num[i:i+12], pos_edge_index[i:i+12], pos_edge_weigts[i:i+12])
            auc, ap = model.test(z, pos_edge_index[i+12], neg_edge_index[i+12])
            
            total_auc += auc
            total_ap += ap
            
    return total_auc/2, total_ap/2

In [17]:
for epoch in range(epochs):
    loss = train()
    
    auc, ap = test(pos_edge_index, neg_edge_index)
    print('Epoch: {:03d}, Train Loss:{:.4f}, AUC: {:.4f}, AP: {:.4f}'.format(epoch+1, loss, auc, ap))

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 001, Train Loss:1.1283, AUC: 0.9557, AP: 0.7938


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 002, Train Loss:0.9245, AUC: 0.9551, AP: 0.7910


  0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch: 027, Train Loss:0.2990, AUC: 0.9820, AP: 0.9075


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 028, Train Loss:0.2827, AUC: 0.9823, AP: 0.9061


100%|██████████| 10/10 [00:03<00:00,  2.66it/s]


KeyboardInterrupt: 

In [18]:
model.encoder.eval()
torch.save(model.encoder.state_dict(), weights_path)

In [ ]:
z.shpae